In [1]:
import os

In [2]:
for root, dirs, files in os.walk("."):
    for filename in files:
        print(filename)

archive (5).zip
Clean_orders_dataset.ipynb
data_with_proposed_data.csv
Filtered_data.csv
HRhd2Y0.png
hs_err_pid2648.log
merged_data_pyspark_with_zip.csv
merged_data_pyspark_zip_distance.csv
merged_data_pyspark_zip_distance_cal_days.csv
merged_data_pyspark_zip_distance_cal_days.cvs
merged_pyspark.csv
merging_data_pyspark.ipynb
Multiple Connections_olist_orders_dataset.csv
olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_order_reviews_dataset_batch4_updated.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
Predicting Delivery Time of Components in a Supply Chain.pdf
product_category_name_translation.csv
replay_pid2648.log
review sentiment analysys.ipynb
reviews_dataset_modified.csv
reviews_dataset_modified_batch4_10.csv
reviews_dataset_modified_batch4_11.csv
review_sentiment_analysis_V_2.ipynb
SVM_model - Copy.ipynb
TimeError.ipynb
Untitled.ipynb
U

In [3]:
import pandas as pd
import numpy as np

In [4]:
customers_df = pd.read_csv('olist_customers_dataset.csv')
geolocation_df = pd.read_csv('olist_geolocation_dataset.csv')
orders_df = pd.read_csv('olist_orders_dataset.csv')
order_items_df = pd.read_csv('olist_order_items_dataset.csv')
order_payments_df = pd.read_csv('olist_order_payments_dataset.csv')
order_reviews_df = pd.read_csv('olist_order_reviews_dataset.csv')
products_df = pd.read_csv('olist_products_dataset.csv')
sellers_df = pd.read_csv('olist_sellers_dataset.csv')
category_name_df = pd.read_csv('product_category_name_translation.csv')

In [5]:
orders_df.describe()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,c99ca23474e04e001665cd9439b59dca,b38fc5ae2f4598b70bc24505a0d818a0,delivered,2018-03-31 15:08:21,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-05-08 23:38:46,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522


In [6]:
orders_df['order_status'].describe()

count         99441
unique            8
top       delivered
freq          96478
Name: order_status, dtype: object

In [7]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [8]:
type(orders_df['order_purchase_timestamp'][0])

str

In [9]:
orders_df[['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']] = orders_df[['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']].astype('datetime64[ns]')
(orders_df['order_purchase_timestamp'][0].date() - orders_df['order_purchase_timestamp'][1].date()).days


-295

In [10]:
delivered_df = orders_df[orders_df['order_status']=='delivered']

In [11]:
#delivered_df['order_delivered_carrier_date'].isnull().values.any()   #.isnull().values.any()
delivered_df['order_delivered_carrier_date'].isnull().values.ravel().sum()

2

In [12]:
#delivered_df['order_delivered_customer_date'].isnull().values.any()
delivered_df['order_delivered_customer_date'].isnull().values.ravel().sum()

8

In [13]:
#delivered_df['order_delivered_customer_date'].isnull().values.any()
delivered_df['order_delivered_customer_date'].isnull().values.ravel().sum()

8

In [14]:
len(delivered_df.index)

96478

In [15]:
delivered_df = delivered_df.dropna(how='any',axis=0)

In [16]:
len(delivered_df.index)

96455

In [17]:
#calculation carrier to customer
delivered_df['carrier-customer'] = delivered_df['order_delivered_customer_date'] - delivered_df['order_delivered_carrier_date']

In [18]:
delivered_df.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,carrier-customer
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,6 days 01:30:13
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,12 days 00:56:45


In [19]:
def days(x):
    x = (x / np.timedelta64(1,'D')) #.astype(int)
    return int(x)
#days(delivered_df['carrier-customer'][10])

In [20]:
delivered_df['carrier-customer'] = delivered_df['carrier-customer'].apply(days)

In [21]:
delivered_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,carrier-customer
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,6
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,12
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,9
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1


In [22]:
#calculation approval to carrier 
delivered_df['carrier-approval'] = delivered_df['order_delivered_carrier_date'] - delivered_df['order_approved_at']

In [23]:
delivered_df['carrier-approval'] = delivered_df['carrier-approval'].apply(days)

In [24]:
delivered_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,carrier-customer,carrier-approval
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,6,2
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,12,0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,9,3
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1,0


In [25]:
#calculation customer to estimate
delivered_df['estimate-customer'] = delivered_df['order_estimated_delivery_date'] - delivered_df['order_delivered_customer_date']

In [26]:
delivered_df['estimate-customer'] = delivered_df['estimate-customer'].apply(days)

In [27]:
delivered_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,carrier-customer,carrier-approval,estimate-customer
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,6,2,7
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,12,0,5
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,0,17
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,9,3,12
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1,0,9
